In [56]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(doMC)
library(lubridate)
library(ggplot2)
library(lfe)
library(ggsci)
library(xgboost)
library(doMC)
registerDoMC(24)

In [25]:
fp  <- '/pool001/mfzhao/'
df  <- read_rds(str_c(fp, 'PROCESSED_DATA/panel_xgr.RDS'))
pp  <- read_csv(str_c(fp, 'PROCESSED_DATA/policyPeriods.csv'))
WM  <- read_rds(str_c(fp, 'PROCESSED_DATA/stateWM.RDS'))

Parsed with column specification:
cols(
  key = col_character(),
  p1sdp = col_date(format = ""),
  p2shp = col_date(format = ""),
  p3rop = col_date(format = "")
)



In [26]:
weightedAlters <- function(df, wm, ...) {
    df %>% 
        select(date, key, ...) %>%
        spread(key = key, value = ...) %>%
        ungroup() %>%
        select(-date) %>%
        as.matrix() -> txn_data
    
    df %>%
        ungroup() %>%
        select(date) %>%
        distinct() %>%
        arrange(date) -> dates
    
    outMatrix <- tcrossprod(txn_data, wm)
    colnames(outMatrix) <- colnames(txn_data)
    
    data.frame(dates, outMatrix) %>%
        gather(key = 'key', value = 'value', -date) %>%
        arrange(date, key) %>%
        select(-date, -key) -> out_df
    return(out_df$value)
}

In [27]:
pp %>%
    mutate(cluster = str_sub(key, 1, 2)) %>%
    select(-key) %>%
    distinct() %>%
    rename(p1date = p1sdp, 
           p2date = p2shp,
           p3date = p3rop) -> statePolicy

In [31]:
folds <- read_rds(str_c(fp, 'PROCESSED_DATA/folds.RDS'))
rdf   <- read_rds(str_c(fp, 'PROCESSED_DATA/residualizer_data.RDS'))

In [50]:
get_coefs <- function(dv, vars, df.r, ...) {
    lhs <- str_c(dv, ' ~ ')
    rhs <- str_c(vars, '| key + date | 0 | cluster')
    form <- as.formula(str_c(lhs, rhs))
    
    model <- felm(form, df.r, weights = df.r$n, ...)
    
    as.data.frame(summary(model)$coef) %>%
        mutate(var   = rownames(.),
               dv    = dv,
               model = ifelse(vars == ' p1sdp.r + p2shp.r + p3rop.r ' , 'base', 'ap')) %>%
        filter(!str_detect(var, '[pPtT][rRmM][cCaA][pPxX]')) %>%
        select(7, 6, 5, 1, 2, 3, 4) -> out
    
    colnames(out) <- c('model', 'dv', 'var', 'estimate', 'se', 't', 'p-val')
    return(out)
}

In [47]:
XGresidualizer <- function(Y, colname) {
    print(colname)
    rdf %>%
        ungroup() %>% 
        mutate(Y = Y,
               Y.r = felm(Y ~ 0 | key + date, ., weights = rdf$n)$resid) -> temp_df
  
    folds <- list(which(temp_df$fold %in% 1), 
                  which(temp_df$fold %in% 2), 
                  which(temp_df$fold %in% 3))

    dm    <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.r_fe + TMAX.r_fe, temp_df), label = temp_df$Y.r)
    param <- list(max_depth=2, eta=.5, silent=1, objective='reg:linear')
    fit   <- xgb.cv(params = param, 
                    data = dm, 
                    folds = folds,
                    nrounds = 100, 
                    early_stopping_rounds = 3, 
                    weight = temp_df$n, 
                    nthread = 1)
    best_n <- fit$best_iteration
    for (i in 1:3) {
        tr  <- temp_df %>% filter(fold != i)
        trm <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.r_fe + TMAX.r_fe, tr), label = tr$Y.r)
        fit <- xgb.train(params = param, 
                         data = trm, 
                         nrounds = best_n, 
                         weight = tr$n,
                         nthread = 1)
        te  <- temp_df %>% filter(fold == i)
        tem <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.r_fe + TMAX.r_fe, te), label = te$Y.r)
        te %>%
            select(date, key) %>%
            mutate(pred = predict(fit, newdata = tem)) -> pred_df 
        assign(str_c('temp',i), pred_df) %>%
        select(-pred, -date, -key)
    }
    out <- bind_rows(temp1, temp2, temp3) %>%
        arrange(date, key) %>%
        mutate(tempname = temp_df$Y.r - pred) %>%
        select(-pred, -date, -key)
    
    colnames(out) <- str_c(colname, '.r')
    return(out)
}

In [53]:
fri <- function(df, iter) {
    df %>%
        select(-matches('p1sdp'),
               -matches('p2shp'),
               -matches('p3rop')) %>%
        left_join(statePolicy %>%
                  mutate(cluster = sample(cluster, n(), replace = F))) %>%
        mutate(p1sdp = as.numeric(date - p1date >= 0),
               p2shp = ifelse(is.na(p2date), 0, as.numeric(date - p2date >= 0)),
               p3rop = as.numeric(date - p3date >= 0)) %>%
        mutate(stalter_p1sdp =  weightedAlters(., WM, p1sdp),
               stalter_p2shp =  weightedAlters(., WM, p2shp),
               stalter_p3rop =  weightedAlters(., WM, p3rop)) -> temp
    
        cols_to_xgr <- colnames(
            temp %>% select(
                p1sdp,
                p2shp,
                p3rop,
                stalter_p1sdp,
                stalter_p2shp,
                stalter_p3rop
            )
        )

    foreach(i = 1:length(cols_to_xgr), .combine = cbind) %do% 
        XGresidualizer(temp[[cols_to_xgr[i]]], cols_to_xgr[i]) -> xg.residuals
    
    temp %>%
        select(-p1sdp,
               -p2shp,
               -p3rop,
               -stalter_p1sdp,
               -stalter_p2shp,
               -stalter_p3rop) %>%
        bind_cols(xg.residuals) -> df.r
    
    ldvs <- c('log_mcbgv.r', 'log_pgt2kmt.r', 'log_pgt1hafh.r', 'log_pnchd.r')
    f1 <- ' p1sdp.r + p2shp.r + p3rop.r '  
    f2 <- ' p1sdp.r + p2shp.r + p3rop.r + stalter_p1sdp.r + stalter_p2shp.r + stalter_p3rop.r '
    
    foreach(dv = ldvs, .combine = rbind) %:% 
        foreach(f = c(f1, f2), .combine = rbind) %do%
        get_coefs(dv, f, df.r) %>%
        select(-se, -t) %>%
        mutate(iter = iter) -> coefs
    
    return(coefs)
}

In [57]:
foreach(i = 1:500, .combine = rbind) %dopar% fri(df, i) -> fri_results

In [61]:
ldvs <- c('log_mcbgv.r', 'log_pgt2kmt.r', 'log_pgt1hafh.r', 'log_pnchd.r')
f1   <- ' p1sdp.r + p2shp.r + p3rop.r '  
f2   <- ' p1sdp.r + p2shp.r + p3rop.r + stalter_p1sdp.r + stalter_p2shp.r + stalter_p3rop.r '

foreach(dv = ldvs, .combine = rbind) %:% 
    foreach(f = c(f1, f2), .comb ine = rbind) %dopar%
    get_coefs(dv, f, df) -> coefs

In [137]:
coefs %>%
    filter(model == 'ap') %>%
    mutate(dv  = str_sub(dv, 1, -3),
           var = str_sub(var, 1, -3)) -> base

options(repr.plot.width=15, repr.plot.height=10)
fri_results %>%
    filter(model == 'ap') %>%
    mutate(dv  = str_sub(dv, 1, -3),
           var = str_sub(var, 1, -3)) %>%
    mutate(estimate = ifelse(var != 'p1sdp', estimate, estimate)) %>%
    ggplot(aes(x = estimate, fill = dv)) + 
    geom_histogram(bins = 40) + 
    geom_vline(aes(xintercept = estimate), size = 1, data = base, color = 'black', inherit.aes = F) +
    facet_grid(dv ~ var, scales = 'free') +
    xlab('') + 
    ylab('') +
    scale_fill_d3() +
    theme_light() + 
    theme(text = element_text(size = 20),
          legend.position = 'bottom') -> p

ggsave('/home/mfzhao/SI_plots/rc3a_fri_main.pdf', p, device = 'pdf', width = 6.5, height = 6.5, scale = 2)

Warning message:
“Ignoring unknown parameters: inherit.aes”


In [135]:
fri_results %>%
    filter(model == 'ap') %>%
    mutate(dv  = str_sub(dv, 1, -3),
           var = str_sub(var, 1, -3)) %>%
    mutate(estimate = ifelse(var != 'p1sdp', estimate, estimate)) %>%
    left_join(base %>% select(model, dv, var, base = estimate)) %>%
    group_by(model, dv, var) %>%
    summarize(pv = sum(estimate > base)/500) %>%
    mutate(pv = ifelse(pv > 0.5, (1 - pv), pv)) %>%
    spread(key = dv, value = pv) %>%
    ungroup() %>%
    select(var, log_mcbgv, log_pgt2kmt, log_pgt1hafh, log_pnchd)

Joining, by = c("model", "dv", "var")



var,log_mcbgv,log_pgt2kmt,log_pgt1hafh,log_pnchd
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
p1sdp,0.092,0.126,0.254,0.380
p2shp,0.000,0.000,0.002,0.000
p3rop,0.002,0.006,0.008,0.010
stalter_p1sdp,0.018,0.024,0.046,0.074
stalter_p2shp,0.032,0.028,0.032,0.022
stalter_p3rop,0.000,0.000,0.000,0.000
